In [1]:
import_statements(ZZ)

# ** Warning **: several names for that object: Z, ZZ
from sage.rings.integer_ring import Z


In [43]:
from itertools import combinations_with_replacement

from ehrhart_quasi_polynomial import get_period, get_gcd, create_polytope_from_matrix
from ehrhart_quasi_polynomial import ehrhart_quasi_polynomial as ehr
from ehrhart_quasi_polynomial import PiecewiseEhrhartQuasiPolynomial as PEQP

from sage.interfaces.gfan import gfan
from sage.matrix.constructor import Matrix as create_matrix
from sage.rings.polynomial.groebner_fan import PolyhedralFan

In [35]:
A = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
PEQP = PiecewiseEhrhartQuasiPolynomial
p = PEQP(A)

In [56]:
def _compute_piecewise(self):
        num_variables = self._A.nrows()
        max_degree = num_variables - self._sec_fan.lineality_dim()

        R = PolynomialRing(QQ, "x", num_variables)
        x_vars = R.gens()

        for idx, cone_dict in enumerate(self._cone_dicts):
            cone_rays = cone_dict["rays"]

            red_cone, dimensions_removed = self._remove_zero_dimensions(cone_rays)
            actual_num_variables = num_variables - len(dimensions_removed)

            needed_points = factorial(actual_num_variables + max_degree)//(
                factorial(actual_num_variables)*factorial(max_degree) )

            interpolation_variables = [x for dim, x in enumerate(x_vars)
                                       if dim not in dimensions_removed]
            T = PolynomialRing(QQ, names=interpolation_variables)

            scaled_rays_removed = [free_module_element(p for dim, p in enumerate(ray)
                                                       if dim not in dimensions_removed)
                                   for ray in cone_dict["scaled_rays"]]
            cone_points = self._generate_cone_points(scaled_rays_removed,
                                                     needed_points)

            polynomials = {}
            for off_set in cone_dict["quotient"]:
                lifted = off_set.lift()
                lifted = free_module_element(off for dim, off in enumerate(lifted)
                                             if dim not in dimensions_removed)
                off_cone_points = [p + lifted + scaled_rays_removed[1] for p in cone_points]

                num_integral_points = []
                for point in off_cone_points:
                    polytope = self._create_polytope_from_matrix(point)
                    num_integral_points.append(len(polytope.integral_points()))

                #return max_degree, off_cone_points, num_integral_points
                interpolated = T.interpolation(max_degree, off_cone_points,
                                               num_integral_points)
                polynomials[tuple(lifted)] = interpolated

                # off_set_poly = interpolated(*self._projection(x_vars))
                # polynomials[tuple(lifted)] = off_set_poly
            self._cone_dicts[idx]["pre_projected"] = polynomials
            # self._cone_dicts[idx]["lifted"] = polynomials

In [63]:
# TODO: make this work using the lattices
def _find_offset(self, point, cone_idx):

        projected = self._projected_coordinates(point)
        off_set = sum( (projected[k] - int(projected[k]))*o_vec
                      for k, o_vec in enumerate(self._scaled_rays[cone_idx]))
        return tuple(int(val) for val in off_set)

def __call__(self, point):
        return self.evaluate(point)

def evaluate(self, point):
        if len(point) != self._amb_dim:
            raise ValueError("Dimension of ``point`` needs to be equal to the ambient"
                             f" dimension of ``self`` which is {self._amb_dim}")

        proj = self._projection(point)
        for idx, cone in enumerate(self._max_cones):
            if proj in cone:

                off_set = _find_offset(point, idx)
                return off_set
                # return self._qps[idx][off_set](point)
        return 0

In [57]:
_compute_piecewise(p)

In [62]:
p._cone_dicts[1]["pre_projected"]

{(0, 0, 0, 0): 5/2*x2^2 + 5*x2*x3 + 7/2*x2 + 2*x3 + 1,
 (0, 1, 0, 1): 5/2*x2^2 + 5*x2*x3 + 7/2*x2 + 2*x3 + 1,
 (0, 2, 0, 2): 5/2*x2^2 + 5*x2*x3 + 7/2*x2 + 2*x3 + 1,
 (0, 3, 0, 3): 5/2*x2^2 + 5*x2*x3 + 7/2*x2 + 2*x3 + 1,
 (0, 4, 0, 4): 5/2*x2^2 + 5*x2*x3 + 7/2*x2 + 2*x3 + 1}

In [64]:
p.evaluate([0, 10, 0, 10])

AttributeError: 'PiecewiseEhrhartQuasiPolynomial' object has no attribute '_max_cones'